<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 810ms/step - loss: 0.6724 - accuracy: 0.5535 - val_loss: 0.6407 - val_accuracy: 0.5756
Epoch 2/10
26/26 [==============================] - 20s 769ms/step - loss: 0.6253 - accuracy: 0.6521 - val_loss: 0.5892 - val_accuracy: 0.7317
Epoch 3/10
26/26 [==============================] - 20s 762ms/step - loss: 0.5377 - accuracy: 0.7676 - val_loss: 0.4859 - val_accuracy: 0.8049
Epoch 4/10
26/26 [==============================] - 20s 769ms/step - loss: 0.4762 - accuracy: 0.8090 - val_loss: 0.4121 - val_accuracy: 0.8585
Epoch 5/10
26/26 [==============================] - 20s 773ms/step - loss: 0.4150 - accuracy: 0.8321 - val_loss: 0.3531 - val_accuracy: 0.8927
Epoch 6/10
26/26 [==============================] - 20s 769ms/step - loss: 0.3612 - accuracy: 0.8528 - val_loss: 0.2848 - val_accuracy: 0.9122
Epoch 7/10
26/26 [==============================] - 20s 769ms/step - loss: 0.3017 - accuracy: 0.8917 - val_loss: 0.2461 - val_accuracy: 0.9268

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.67.h5
26/26 - 21s - loss: 0.6765 - accuracy: 0.5584 - val_loss: 0.6730 - val_accuracy: 0.5171
Epoch 2/5

Epoch 00002: saving model to weights.02-0.63.h5
26/26 - 20s - loss: 0.6461 - accuracy: 0.6253 - val_loss: 0.6282 - val_accuracy: 0.5805
Epoch 3/5

Epoch 00003: saving model to weights.03-0.67.h5
26/26 - 20s - loss: 0.6138 - accuracy: 0.6727 - val_loss: 0.6692 - val_accuracy: 0.5366
Epoch 4/5

Epoch 00004: saving model to weights.04-0.68.h5
26/26 - 20s - loss: 0.5723 - accuracy: 0.7080 - val_loss: 0.6803 - val_accuracy: 0.5561
Epoch 5/5

Epoch 00005: saving model to weights.05-0.47.h5
26/26 - 20s - loss: 0.5355 - accuracy: 0.7397 - val_loss: 0.4708 - val_accuracy: 0.8683


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 22s - loss: 0.6733 - accuracy: 0.5973 - val_loss: 0.6891 - val_accuracy: 0.4390


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 21s - loss: 0.6808 - accuracy: 0.5876 - val_loss: 0.6610 - val_accuracy: 0.8098
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6429 - accuracy: 0.6873 - val_loss: 0.6399 - val_accuracy: 0.6244


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6559 - accuracy: 0.6533 - val_loss: 0.6892 - val_accuracy: 0.4683
Epoch 2/50
26/26 - 20s - loss: 0.5920 - accuracy: 0.7105 - val_loss: 0.5572 - val_accuracy: 0.7902
Epoch 3/50
26/26 - 20s - loss: 0.5393 - accuracy: 0.7567 - val_loss: 0.4812 - val_accuracy: 0.8000
Epoch 4/50
26/26 - 20s - loss: 0.4469 - accuracy: 0.8248 - val_loss: 0.4451 - val_accuracy: 0.8293
Epoch 5/50
26/26 - 20s - loss: 0.4044 - accuracy: 0.8406 - val_loss: 0.4343 - val_accuracy: 0.8049
Epoch 6/50
26/26 - 20s - loss: 0.3230 - accuracy: 0.8759 - val_loss: 0.2626 - val_accuracy: 0.9317
Epoch 7/50
26/26 - 20s - loss: 0.2530 - accuracy: 0.9246 - val_loss: 0.2094 - val_accuracy: 0.9463
Epoch 8/50
26/26 - 20s - loss: 0.2079 - accuracy: 0.9270 - val_loss: 0.1602 - val_accuracy: 0.9561
Epoch 9/50
26/26 - 20s - loss: 0.1659 - accuracy: 0.9513 - val_loss: 0.1343 - val_accuracy: 0.9756
Epoch 10/50
26/26 - 20s - loss: 0.1519 - accuracy: 0.9623 - val_loss: 0.0993 - val_accuracy: 0.9854
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 20s 776ms/step - loss: 0.6822 - accuracy: 0.5365 - val_loss: 0.6485 - val_accuracy: 0.7561
Epoch 2/5
26/26 [==============================] - 20s 780ms/step - loss: 0.6384 - accuracy: 0.6521 - val_loss: 0.6652 - val_accuracy: 0.4780
Epoch 3/5
26/26 [==============================] - 20s 763ms/step - loss: 0.5831 - accuracy: 0.7190 - val_loss: 0.5462 - val_accuracy: 0.8000
Epoch 4/5
26/26 [==============================] - 20s 768ms/step - loss: 0.5178 - accuracy: 0.7762 - val_loss: 0.5336 - val_accuracy: 0.7171
Epoch 5/5
26/26 [==============================] - 20s 769ms/step - loss: 0.4694 - accuracy: 0.7932 - val_loss: 0.4910 - val_accuracy: 0.7171


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.536496,0.682415,0.756098,0.648486
1,1,0.652068,0.639019,0.478049,0.665246
2,2,0.718978,0.584175,0.800000,0.546182
3,3,0.776156,0.518010,0.717073,0.533577
4,4,0.793187,0.467320,0.717073,0.491006


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 29s 1s/step - loss: 0.6833 - accuracy: 0.5523 - val_loss: 0.6806 - val_accuracy: 0.4927

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 20s 776ms/step - loss: 0.6616 - accuracy: 0.6217 - val_loss: 0.6719 - val_accuracy: 0.5122

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 20s 773ms/step - loss: 0.6502 - accuracy: 0.6229 - val_loss: 0.6604 - val_accuracy: 0.5805

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 20s 776ms/step - loss: 0.6452 - accuracy: 0.6460 - val_loss: 0.6584 - val_accuracy: 0.5756

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 21s 793ms/step - loss: 0.6415

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [ ]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 835ms/step - loss: 0.6702 - accuracy: 0.5864 - val_loss: 0.6577 - val_accuracy: 0.5951
Epoch 2/50
26/26 [==============================] - 21s 792ms/step - loss: 0.6259 - accuracy: 0.7129 - val_loss: 0.5979 - val_accuracy: 0.7854
Epoch 3/50
26/26 [==============================] - 20s 788ms/step - loss: 0.5703 - accuracy: 0.7713 - val_loss: 0.5296 - val_accuracy: 0.8293
Epoch 4/50
26/26 [==============================] - 20s 777ms/step - loss: 0.5052 - accuracy: 0.7956 - val_loss: 0.5073 - val_accuracy: 0.7854
Epoch 5/50
26/26 [==============================] - 21s 792ms/step - loss: 0.4181 - accuracy: 0.8491 - val_loss: 0.3681 - val_accuracy: 0.8683
Epoch 6/50
26/26 [==============================] - 21s 792ms/step - loss: 0.3826 - accuracy: 0.8528 - val_loss: 0.3099 - val_accuracy: 0.9122
Epoch 7/50
26/26 [==============================] - 20s 769ms/step - loss: 0.2901 - accuracy: 0.9161 - val_loss: 0.2590 - val_accuracy: 0.9220

In [ ]:
%tensorboard --logdir log_dir